In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

E:\anaconda\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
photos = np.load('citra_sel_darah_putih.npy')

In [4]:
labels = pd.read_excel('label_darah.xlsx')

In [5]:
labels = labels.drop('Unnamed: 0', axis=1)

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_label = onehot_encoder.fit_transform(integer_encoded)
y_label[:5]

[0 0 0 ... 5 5 5]


E:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(photos, y_label, test_size=0.3, random_state=0)

In [18]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [19]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 135424)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)               

In [22]:
train = X_train/X_train.max()
X_test = X_test/X_train.max()

In [23]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train, y_train, epochs=100, validation_data=(X_test, y_test))

Train on 1730 samples, validate on 742 samples
Epoch 1/100


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Can not squeeze dim[1], expected a dimension of 1, got 6
	 [[{{node metrics_10/acc/Squeeze}}]]
	 [[metrics_10/acc/Identity/_231]]
  (1) Invalid argument: Can not squeeze dim[1], expected a dimension of 1, got 6
	 [[{{node metrics_10/acc/Squeeze}}]]
0 successful operations.
0 derived errors ignored.